In [3]:
import sys

sys.path.append("../..")

from data_loading import *


SyntaxError: invalid syntax (merra2.py, line 66)

In [ ]:
n = 100000000
estimate_quantile = 0.9935

means = {"U": 5.589, "V": 0.018}
stds = {"U": 9.832, "V": 3.232}


In [ ]:
data = pd.read_feather(f"UV-{estimate_quantile}-{n}.ft")
df = data.copy()

NORMALIZED = False
GEOGRAPHICAL = False
CYCLIC_TIME = False

data.head()


In [2]:
NORMALIZED = True

df["lat"] = (data["lat"] * 0.5) - 90
df["lat"] /= 90
df["lon"] = (data["lon"] * 0.625) - 180
df["lon"] /= 180
df["lev"] /= 36

df["lev"] = df["lev"].astype("float16")
df["lat"] = df["lat"].astype("float16")
df["lon"] = df["lon"].astype("float16")

df["U"] -= means["U"]
df["U"] /= stds["U"]

df["V"] -= means["V"]
df["V"] /= stds["V"]

df["U_est"] -= means["U"]
df["U_est"] /= stds["U"]

df["V_est"] -= means["V"]
df["V_est"] /= stds["V"]

df["day %"] = (data["time"] % 8) / 8
df["year %"] = data["time"] / 2920
del df["time"]

df.head()


NameError: name 'data' is not defined

In [22]:
CYCLIC_TIME = True

df["day %"] *= np.pi * 2
df["year %"] *= np.pi * 2

df["sin_day"] = np.sin(df["day %"]).astype("float16")
df["cos_day"] = np.cos(df["day %"]).astype("float16")
df["sin_year"] = np.sin(df["year %"]).astype("float16")
df["cos_year"] = np.cos(df["year %"]).astype("float16")

del df["day %"]
del df["year %"]

df.head()


,lev,lat,lon,U_est,V_est,U,V,sin_day,cos_day,sin_year,cos_year
0,0.018524,-0.133301,0.881836,-1.126953,-0.458252,-1.098633,-0.322998,0.707031,-0.707031,0.650391,-0.759766
1,0.012344,-0.327881,0.329834,-0.543945,-0.122070,-0.512695,-0.249146,0.000000,-1.000000,0.280273,0.959961
2,0.005402,0.899902,0.107666,0.445557,1.446289,0.415283,1.588867,-1.000000,-0.000000,-0.689941,-0.723633
3,0.023926,0.033325,-0.843750,-1.445312,-0.211426,-1.453125,-0.103638,-0.707031,-0.707031,0.617188,0.787109
4,0.023148,0.588867,-0.013885,-0.540039,0.079895,-0.569824,0.047821,-1.000000,-0.000000,0.980469,-0.196655


In [19]:
GEOGRAPHICAL = True

frland = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLAND",
                                         time=0, level=0, folder="..").astype("float16")
frocean = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FROCEAN",
                                          time=0, level=0, folder="..").astype("float16")
frlake = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLAKE",
                                         time=0, level=0, folder="..").astype("float16")
frlandice = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="FRLANDICE",
                                            time=0, level=0, folder="..").astype("float16")
phis = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="PHIS",
                                       time=0, level=0, folder="..")
sgh = load_variable_at_time_and_level("MERRA2_101.const_2d_asm_Nx.00000000.nc4", variable="SGH",
                                      time=0, level=0, folder="..")

phis -= phis.mean()
phis /= phis.std()
phis = phis.astype("float16")

sgh -= sgh.mean()
sgh /= sgh.std()
sgh = sgh.astype("float16")

indices = data[["lat", "lon"]].values.T
indices = tuple()

df["frland"] = frland[indices]
df["frocean"] = frocean[indices]
df["frlake"] = frlake[indices]
df["frlandice"] = frlandice[indices]
df["phis"] = phis[indices]
df["sgh"] = sgh[indices]

df.head()


NameError: name 'load_variable_at_time_and_level' is not defined

In [31]:
df.dtypes

lev          float16
lat          float16
lon          float16
U_est        float16
V_est        float16
U            float16
V            float16
sin_day      float16
cos_day      float16
sin_year     float16
cos_year     float16
frland       float16
frocean      float16
frlake       float16
frlandice    float16
phis         float16
sgh          float16
dtype: object

In [32]:
# N - normalized
# G - geographical features
# CT - cyclic time

df.to_feather(f"UV-NGCT-{estimate_quantile}-{n}.ft")

In [33]:
df.astype("float32").describe()


,lev,lat,lon,U_est,V_est,U,V,sin_day,cos_day,sin_year,cos_year,frland,frocean,frlake,frlandice,phis,sgh
count,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08
mean,1.350356e-02,-2.788276e-05,-1.729555e-03,-8.698047e-05,-1.218313e-04,-8.481942e-05,-1.336878e-04,9.476574e-05,4.270496e-05,4.427868e-05,-2.518046e-05,2.208237e-01,6.578948e-01,4.558672e-03,1.167235e-01,-6.161448e-05,5.997979e-05
std,8.015768e-03,5.789430e-01,5.773962e-01,9.994370e-01,9.930584e-01,1.000275e+00,9.997646e-01,7.070808e-01,7.070572e-01,7.071519e-01,7.070696e-01,3.984058e-01,4.615981e-01,2.931167e-02,3.150467e-01,9.998299e-01,1.000197e+00
min,0.000000e+00,-1.000000e+00,-1.000000e+00,-5.382812e+00,-8.187500e+00,-5.367188e+00,-8.281250e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.087891e-01,-3.937988e-01
25%,6.942749e-03,-5.000000e-01,-5.034180e-01,-6.630859e-01,-5.639648e-01,-6.630859e-01,-5.683594e-01,-7.070312e-01,-7.070312e-01,-7.070312e-01,-7.070312e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.477539e-01,-3.937988e-01
50%,1.388550e-02,0.000000e+00,-3.471375e-03,-2.152100e-01,-1.185608e-02,-2.144775e-01,-1.174164e-02,0.000000e+00,-0.000000e+00,0.000000e+00,-0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-4.477539e-01,-3.937988e-01
75%,2.082825e-02,5.000000e-01,5.000000e-01,4.860840e-01,5.483398e-01,4.868164e-01,5.527344e-01,7.070312e-01,7.070312e-01,7.070312e-01,7.070312e-01,9.777832e-02,1.000000e+00,0.000000e+00,0.000000e+00,-1.679688e-01,-1.479492e-01
max,2.700806e-02,1.000000e+00,9.965820e-01,7.992188e+00,8.617188e+00,8.031250e+00,8.679688e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.078125e+00,1.228906e+01
